In [27]:
# Import the libraries
import os
import numpy as np
import csv
import pandas as pd
import multiprocessing as mp
import shutil
import docker

#docker_client = docker.from_env()

In [28]:
# List with all the filenames to search
FILNAMES = [
    "x_sobol.csv",
    "x_morris.csv",
    "x_lhs.csv",
    "y_sobol.csv",
    "y_morris.csv",
    "y_lhs.csv"
]

In [29]:
# Set the reference intrusion
DELTA_STAR = 120*0.70

In [30]:
# SET A TOTAL BUDGET
TOTAL_BUDGET:int = 200

#### Definition of the Objective Function
$$f(\mathbf{x} ) = \text{SEA} (\mathbf{x}) - \lambda | \delta(\mathbf{x}) - \delta^{\star}  |$$

In [31]:
# Definition of the objective function
#f = lambda SEA,intrusion,lamda: SEA - np.multiply(lamda,np.abs(intrusion-DELTA_STAR))

f = lambda SEA,intrusion,lamda: SEA - np.multiply(lamda,np.maximum(intrusion-DELTA_STAR,0))

In [32]:
# Get the root directories to the SEA and Intrusion Datasets
print(os.getcwd())

# Change the working directory
os.chdir('/home/ivanolar/Documents')

parent_dir = os.path.dirname(os.getcwd())



/home/ivanolar/Documents


In [33]:
# Set the intrusion and SEA storing directories
intrusion_dir = os.path.join(os.getcwd(),"data-Intrusion")
sea_dir = os.path.join(os.getcwd(),"data-SEA")

data_dir =  os.path.join(os.getcwd(),"data")
output_dir = os.path.join(os.getcwd(),"output")

# Print all the paths to ensure are well written
print(intrusion_dir,os.path.exists(intrusion_dir))
print(sea_dir,os.path.exists(sea_dir))
print(data_dir,os.path.exists(data_dir))
print(output_dir,os.path.exists(output_dir))


/home/ivanolar/Documents/data-Intrusion True
/home/ivanolar/Documents/data-SEA True
/home/ivanolar/Documents/data True
/home/ivanolar/Documents/output True


In [34]:
# Generate an array of Numpy arrays to read
input_data_files = []
for ii in range(0,3):
    curfil_1 = os.path.join(intrusion_dir ,FILNAMES[ii])
    input_data_files.append(np.loadtxt(curfil_1))
    
    
    

In [35]:
# Generate an array of Numpy arrays to read
intrusion_files_array = list()
sea_files_array = list()
for ii in range(3,6):
    curfil_1 = os.path.join(intrusion_dir ,FILNAMES[ii])
    curfil_2 = os.path.join(sea_dir ,FILNAMES[ii])
    intrusion_files_array.append(np.loadtxt(curfil_1))
    sea_files_array.append(np.loadtxt(curfil_2))
    
    

In [36]:
# Generate and array to check how the lamda parameter "affects the landscape"
num_iterations:int = 1
lamda_array = [763]
#num_iterations = 4
#lamda_array = np.array([450,465,480,472.5])

In [37]:
print(lamda_array)

[763]


In [38]:
# Get arrays which store the different functions
mixed_function_sobol = list()
mixed_function_morris = list()
mixed_function_lhs = list()

In [39]:
# Start a loop to evaluate the "synthetic objective function"
for ii in range(num_iterations):
    
    # Compute the synthetic sobol
    sobol_synth = f(SEA=sea_files_array[0],intrusion=intrusion_files_array[0],lamda=lamda_array[ii])
    
    # Get the Morris synthetic computation
    morris_synth = f(SEA=sea_files_array[1],intrusion=intrusion_files_array[1],lamda=lamda_array[ii])

    # Get the LHS synthetic computation
    lhs_synth = f(SEA=sea_files_array[2],intrusion=intrusion_files_array[2],lamda=lamda_array[ii])

    # Get the x data from Sobol, Morris and LHS Sampling
    x_sobol = input_data_files[0]
    x_morris = input_data_files[1]
    x_lhs = input_data_files[2]

    # Get the extremal values of x (from Morris)
    idxs_1 =((x_morris == 5) | (x_morris == -5)).all(axis=1)

    x_morris_fixed = x_morris[idxs_1]
    y_morris_fixed = morris_synth[idxs_1]

    x_database:np.ndarray = np.vstack((x_sobol,x_lhs))
    y_database:np.ndarray = np.hstack((sobol_synth,lhs_synth))

    y_morris_fixed_intrusion:np.ndarray = intrusion_files_array[1][idxs_1]
    y_morris_fixed_sea:np.ndarray = sea_files_array[1][idxs_1]

    y_database_instrusion:np.ndarray = np.hstack((intrusion_files_array[0],intrusion_files_array[2]))
    y_database_sea:np.ndarray = np.hstack((sea_files_array[0],sea_files_array[2]))

    # Get the indices to get the samples
    indices_ = np.random.choice(y_database.size, size=TOTAL_BUDGET-y_morris_fixed.size
                                , replace=False)
    
    x_exp:np.ndarray = np.vstack((x_morris_fixed,x_database[indices_,:]))
    y_exp:np.ndarray = np.hstack((y_morris_fixed,y_database[indices_]))

    y_exp_intrusion:np.ndarray = np.hstack((y_morris_fixed_intrusion,y_database_instrusion[indices_]))
    y_exp_sea:np.ndarray = np.hstack((y_morris_fixed_sea,y_database_sea[indices_]))
    
    print(indices_)
    # Go to the data folder and delete all the files
    for directory, dirnams , filnams in os.walk(data_dir):
        for nam in filnams:
            if (nam.endswith('.csv')):
                # nam.endswith('y_sobol.csv') or  
                # nam.endswith('y_morris.csv') or 
                # nam.endswith('y_lhs.csv') or 
                # nam.endswith('y.csv') or
                # nam.endswith('x_sobol.csv') or  
                # nam.endswith('x_morris.csv') or 
                # nam.endswith('x_lhs.csv') or 
                # nam.endswith('x.csv')):
                os.remove(os.path.join(data_dir,nam))

    # Save the files in the data directory
    #np.savetxt(fname=os.path.join(data_dir,'y_sobol.csv'),X=sobol_synth)
    #np.savetxt(fname=os.path.join(data_dir,'y_morris.csv'),X=morris_synth)
    #np.savetxt(fname=os.path.join(data_dir,'y_lhs.csv'),X=lhs_synth)

    # Save the X data
    #np.savetxt(fname=os.path.join(data_dir,'x_sobol.csv'),X=x_sobol)
    #np.savetxt(fname=os.path.join(data_dir,'x_morris.csv'),X=x_morris)
    #np.savetxt(fname=os.path.join(data_dir,'x_lhs.csv'),X=x_lhs)

    # Save the x data
    np.savetxt(fname=os.path.join(data_dir,'x.csv'),X=x_exp)
    np.savetxt(fname=os.path.join(data_dir,'y.csv'),X=y_exp)
    # Save a file with the lamda used
    np.savetxt(fname=os.path.join(data_dir,'lamda.csv'),X=np.array([lamda_array[ii]]))

    # Save the subsampled intrusion and sea
    np.savetxt(fname=os.path.join(data_dir,'sea_y.csv'),X=y_exp_sea)
    np.savetxt(fname=os.path.join(data_dir,'intrusion_y.csv'),X=y_exp_intrusion)

    # Run the GSA Report
    os.system("""docker run --rm -v "$(pwd)"/output:/output -v "$(pwd)"/data:/data     ghcr.io/nikivanstein/gsareport:main -p /data/problem.json -d /data -o /output""")

    # Copy the data file into a new directory
    new_dir:str = os.path.join(os.getcwd(),f"data_{str(ii+1)}_mod")
    if os.path.exists(new_dir):
        shutil.rmtree(new_dir,ignore_errors=True)

    shutil.copytree(data_dir,new_dir)

    # Copy the data file into a new directory
    new_dir:str = os.path.join(os.getcwd(),f"output_{str(ii+1)}_mod")
    if os.path.exists(new_dir):
        shutil.rmtree(new_dir,ignore_errors=True)

    shutil.copytree(output_dir,new_dir)

    # Delete all the files in the output directory

    # Go to the data folder and delete all the files
    for directory, dirnams, filnams in os.walk(output_dir):
        for nam in filnams:
            if nam.endswith(".html"):
                os.remove(os.path.join(output_dir,nam))
        
    

[ 353  549  941  817  529 1042  412  586 1441  469 1256  224 1352 1519
 1268  486  745 1554  986 1168  845  425 1393 1298  889  686  205  294
  924  934  741 1105  847  416  597  459  848 1401  723  201  385 1178
  910  488 1387   11  738  891  132 1359  377 1297 1459  961  551  434
 1112  907  138 1582 1507  223  736  712  520   32 1403  453 1020  528
 1569  181  386  756 1333  594   88  513  525  902  383 1431   15   87
  476 1418  943  204 1088 1474  911   18 1589  209 1587  199  273 1119
  207  543 1526  242 1517 1015   27  213 1176   82 1011 1310 1390  533
 1397   71  602  932  114 1120  558  399 1230   14  118  154 1351 1428
  524 1155  689   26  415  398 1529 1496  944  276   35  480  633 1448
 1544 1116  539  668 1100 1340  493 1164  837 1438 1511  269  509 1385
 1490  447   96  966 1103  327  392 1377  450  983  920  516  839  304
  455  485  102   83 1139 1395 1068]
loaded problem definition


100%|██████████| 100/100 [00:02<00:00, 42.36it/s]
